## Import & Config

In [11]:
import os
import json
import numpy as np
import torch
import av
import cv2
from transformers import AutoModel, AutoTokenizer
import faiss
import pickle
from torch.utils.tensorboard import SummaryWriter

# ==== CONFIGURATION ====
VIDEO_DIR       = "input_videos"
OUTPUT_DIR      = "videoclip_output"
FRAMES_PER_CLIP = 8
CLIP_DURATION   = 30
BATCH_SIZE      = 128

if torch.cuda.is_available() : 
    device = "cuda:0"
    print(f"device:{device}")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print(f"device:{device}")
else:
    print(f"Plain ol' CPU")

# ==== LOAD ViCLIP ====
model = AutoModel.from_pretrained("OpenGVLab/ViCLIP-L-14-hf", trust_remote_code=True).to(device).eval()

/home/jupyter-dai7591/.conda/envs/dedup/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device:cuda:0


/home/jupyter-dai7591/.conda/envs/dedup/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/jupyter-dai7591/.conda/envs/dedup/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/home/jupyter-dai7591/.cache/huggingface/modules/transformers_modules/OpenGVLab/ViCLIP-L-14-hf/1652361522e1cb41c28cdfae870f690d00e7456b/viclip_text.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  

## Create embeddings

In [1]:
# extract_embeddings.py

import math
import torch
import numpy as np
from decord import VideoReader, cpu
from transformers import AutoModel, CLIPImageProcessor

# ─── 1) Load the model & image processor ───
MODEL_ID = "OpenGVLab/InternVL-14B-224px"
model = AutoModel.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    device_map="auto"
).eval()
image_processor = CLIPImageProcessor.from_pretrained(MODEL_ID)

# ─── 2) Helper to sample & preprocess frames ───
def extract_frames(video_path, num_frames=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    total = len(vr)
    interval = math.ceil(total / num_frames)
    # pick center of each interval
    idxs = (np.arange(0, total, interval) + interval // 2).clip(max=total-1)
    frames = vr.get_batch(idxs).asnumpy()  # shape: [num_frames, H, W, 3]
    return frames

# ─── 3) Main embedding extraction ───
if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("video_path", type=str, help="Path to input video file")
    parser.add_argument("--frames", type=int, default=32, help="How many frames to sample")
    args = parser.parse_args()

    # 3.1 Sample raw frames
    raw_frames = extract_frames(args.video_path, num_frames=args.frames)

    # 3.2 Turn into model inputs
    # image_processor will resize, normalize, and batchify
    batch = image_processor(images=list(raw_frames), return_tensors="pt")
    pixel_values = batch.pixel_values.to(torch.bfloat16).cuda()  # shape [B, 3, H, W]

    # 3.3 Encode & pool
    with torch.no_grad():
        feats = model.encode_image(pixel_values, mode="InternVL-C")  # [B, D]
        # average pooling → one vector per video
        video_embed = feats.mean(dim=0)                           # [D]
        video_embed = torch.nn.functional.normalize(video_embed, dim=-1)

    # 3.4 Detach to NumPy
    emb_np = video_embed.cpu().numpy()
    print(f"Extracted video embedding (shape {emb_np.shape}):")
    print(emb_np)

/home/jupyter-dai7591/.conda/envs/dedup/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL-14B-224px:
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/OpenGVLab/InternVL-14B-224px:
- configuration_internvl.py
- configuration_intern_vit.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Encountered exception while importing peft: No module named 'peft'


ImportError: This modeling file requires the following packages that were not found in your environment: peft. Run `pip install peft`

## Visualize embeddings

In [8]:
# Load lookup metadata
with open("videoclip_output/embedding_lookup.json", "r") as f:
    lookup = json.load(f)

# Ensure entries are sorted by clip_id (assumed to match row index)
sorted_lookup = sorted(lookup.values(), key=lambda x: int(x["clip_id"]))

# Create metadata strings like: "video.mp4[3] @ 90s"
metadata = [
    f'{entry["video_file"]}[{entry["clip_index"]}] @ {entry["start_time_sec"]}s'
    for entry in sorted_lookup
]

# Load embedding matrix
emb_matrix = np.load("videoclip_output/video_embeddings.npy")

# Ensure alignment
assert emb_matrix.shape[0] == len(metadata), (
    f"❌ {emb_matrix.shape[0]} embeddings vs {len(metadata)} metadata entries"
)

# Write to TensorBoard
writer = SummaryWriter(log_dir="runs/embeds")
writer.add_embedding(
    emb_matrix,
    metadata=metadata,
    tag="my_embeddings"
)
writer.close()

print("✅ Done. Run:\n  tensorboard --logdir=runs/embeds\nThen open http://localhost:6006/#projector")

✅ Done. Run:
  tensorboard --logdir=runs/embeds
Then open http://localhost:6006/#projector


## Index in FAISS

In [12]:
EMB_PATH    = "videoclip_output/video_embeddings.npy"
LOOKUP_PATH = "videoclip_output/embedding_lookup.json"
INDEX_PATH  = "videoclip_output/video_embeddings.index"

# Load embeddings and lookup dict
embeddings = np.load(EMB_PATH)  
with open(LOOKUP_PATH, "r") as f:
    lookup = json.load(f)

# Build FAISS index (inner-product) and add IDs
faiss.normalize_L2(embeddings)
dim   = embeddings.shape[1]
index = faiss.IndexFlatIP(dim)
index = faiss.IndexIDMap(index)

ids = np.arange(embeddings.shape[0], dtype="int64")
index.add_with_ids(embeddings, ids)

# Save the index for later
faiss.write_index(index, INDEX_PATH)

## Find duplicates

In [ ]:
from collections import defaultdict
import numpy as np
import json
import faiss

def count_similar_clip_pairs(radius: float):
    """
    For each unique pair of video files, count how many clip pairs are similar
    (i.e., within the given radius), and record the average distance.
    Returns: list of (file1, file2, count, avg_distance)
    """
    embeddings = np.load(EMB_PATH)
    with open(LOOKUP_PATH, "r") as f:
        lookup = json.load(f)

    index = faiss.read_index(INDEX_PATH)
    faiss.normalize_L2(embeddings)

    lims, distances, labels = index.range_search(embeddings, radius)

    # Map (file1, file2) to list of distances
    pair_stats = defaultdict(list)

    for q in range(len(embeddings)):
        start, end = lims[q], lims[q+1]
        for i in range(start, end):
            idx = labels[i]
            if idx <= q:
                continue  # skip self or repeated
            f1 = lookup[str(q)]["video_file"]
            f2 = lookup[str(idx)]["video_file"]
            if f1 != f2:
                key = tuple(sorted((f1, f2)))
                pair_stats[key].append(distances[i])

    results = []
    for (f1, f2), dists in pair_stats.items():
        avg_dist = sum(dists) / len(dists)
        results.append((f1, f2, len(dists), avg_dist))

    return results

# Choose your similarity radius (e.g., 0.8 for cosine similarity >= 0.8)
radius = 0.999
pairs = count_similar_clip_pairs(radius)

for f1, f2, count, avg_dist in sorted(pairs, key=lambda x: -x[2]):
    print(f"{f1} <--> {f2} | similar clips: {count} | avg distance: {avg_dist:.4f}")

TNS_0030_V.mp4 <--> TNS_0031_V.mp4 | similar clips: 205 | avg distance: 1.0000
TNS_0024_V.mp4 <--> TNS_0025_V.mp4 | similar clips: 59 | avg distance: 1.0000


## Evaluate duplicates

In [18]:
import re
import csv

with open("dup_groundtruth.csv", newline="") as f:
    reader = csv.reader(f)
    # if your CSV has a header, skip it:
    next(reader, None)
    # columns: filename, status
    ground_truth = {row[0]: row[1] for row in reader}


def evaluate_duplicates(pairs, ground_truth):
    def extract_id(filename):
        match = re.search(r'TNS_(\d+)', filename)
        return match.group(1) if match else None

    # Normalize detected pairs based on ID number
    detected_duplicates = set()
    for f1, f2, _, _ in pairs:
        id1 = extract_id(f1)
        id2 = extract_id(f2)
        if id1 and id2:
            detected_duplicates.add(tuple(sorted([id1, id2])))

    # Normalize ground-truth: find all DUPLICATE → PRINCIPAL mappings
    true_duplicates = {k for k, v in ground_truth.items() if v.upper() == "DUPLICATE"}
    true_pairs = set()
    for dup in true_duplicates:
        dup_id = extract_id(dup)
        for principal, status in ground_truth.items():
            if status.upper() == "PRINCIPAL":
                principal_id = extract_id(principal)
                if dup_id and principal_id:
                    true_pairs.add(tuple(sorted([dup_id, principal_id])))

    # Evaluation
    true_positives = detected_duplicates & true_pairs
    false_positives = detected_duplicates - true_pairs
    false_negatives = true_pairs - detected_duplicates

    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if (len(true_positives) + len(false_negatives)) > 0 else 0.0

    print(f"✅ True Positives (correctly detected): {len(true_positives)}")
    print(f"❌ False Positives (wrongly flagged):   {len(false_positives)}")
    print(f"🔍 False Negatives (missed duplicates): {len(false_negatives)}")
    print(f"📈 Recall: {recall:.2%}")

    return {
        "true_positives": true_positives,
        "false_positives": false_positives,
        "false_negatives": false_negatives,
        "recall": recall,
    }
eval_result = evaluate_duplicates(pairs, ground_truth)

✅ True Positives (correctly detected): 2
❌ False Positives (wrongly flagged):   0
🔍 False Negatives (missed duplicates): 4685308
📈 Recall: 0.00%


## Find duplicates and optimal radius

In [22]:
import numpy as np
import json
import faiss
import re
import pandas as pd
from collections import defaultdict

# Paths to embeddings, lookup, and index
EMB_PATH    = 'videoclip_output/video_embeddings.npy'
LOOKUP_PATH = 'videoclip_output/embedding_lookup.json'
INDEX_PATH  = 'videoclip_output/video_embeddings.index'

def count_similar_clip_pairs(radius):
    embeddings = np.load(EMB_PATH)
    with open(LOOKUP_PATH) as f:
        lookup = json.load(f)
    index = faiss.read_index(INDEX_PATH)
    faiss.normalize_L2(embeddings)
    lims, distances, labels = index.range_search(embeddings, radius)

    pair_stats = defaultdict(list)
    for q in range(len(embeddings)):
        for i in range(lims[q], lims[q+1]):
            idx = labels[i]
            if idx <= q:
                continue
            f1 = lookup[str(q)]['video_file']
            f2 = lookup[str(idx)]['video_file']
            if f1 != f2:
                pair = tuple(sorted((f1, f2)))
                pair_stats[pair].append(distances[i])

    return [
        (f1, f2, len(dists), sum(dists)/len(dists))
        for (f1, f2), dists in pair_stats.items()
    ]

def evaluate_duplicates(pairs, ground_truth):
    def extract_id(name):
        m = re.search(r'TNS_(\d+)', name)
        return m.group(1) if m else None

    detected = {
        tuple(sorted((extract_id(f1), extract_id(f2))))
        for f1, f2, _, _ in pairs
    }
    truth_dups = {
        extract_id(k)
        for k, v in ground_truth.items()
        if v.upper() == 'DUPLICATE'
    }
    truth_pairs = {
        tuple(sorted((dup, extract_id(pr))))
        for dup in truth_dups
        for pr, st in ground_truth.items()
        if st.upper() == 'PRINCIPAL'
    }

    tp = len(detected & truth_pairs)
    fp = len(detected - truth_pairs)
    fn = len(truth_pairs - detected)
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

    return {
        'true_positives': tp,
        'false_positives': fp,
        'false_negatives': fn,
        'recall': recall
    }

# --- Load ground truth ---
gt_df = pd.read_csv('dup_groundtruth.csv')  # comma-separated file
ground_truth = dict(zip(gt_df['UAR Code'], gt_df['Principal / Duplicate']))

# --- Loop over different radii ---
radii = [0.9, 0.95, 0.99, 0.999]
results = []

for r in radii:
    pairs   = count_similar_clip_pairs(r)
    metrics = evaluate_duplicates(pairs, ground_truth)
    results.append({'radius': r, **metrics})

# --- Export & display ---
df = pd.DataFrame(results)
df.to_csv('evaluation_results.csv', index=False)
print(df)

   radius  true_positives  false_positives  false_negatives        recall
0   0.900               3               28          4685307  6.402991e-07
1   0.950               2               10          4685308  4.268661e-07
2   0.990               2                0          4685308  4.268661e-07
3   0.999               2                0          4685308  4.268661e-07


## Search prompts

In [ ]:
import json
import numpy as np
import faiss
import torch
from transformers import AutoTokenizer, AutoModel

# ─── CONFIG ────────────────────────────────────────────────────────────────────
LOOKUP_PATH = "output/embedding_lookup.json"
INDEX_PATH  = "output/video_embeddings.index"
MODEL_CHECKPOINT = "microsoft/xclip-base-patch32"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# ────────────────────────────────────────────────────────────────────────────────

# Load lookup table and FAISS index
with open(LOOKUP_PATH, "r") as f:
    lookup = json.load(f)
index = faiss.read_index(INDEX_PATH)

# Load tokenizer & text model
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
model = AutoModel.from_pretrained(MODEL_CHECKPOINT).to(DEVICE)
model.eval()

def search_prompts(prompts, top_k=1):
    """
    Encode text prompts, search the FAISS index, and return
    filename + timestamp for each top-k match.
    """
    # Tokenize and encode
    inputs = tokenizer(prompts, padding=True, return_tensors="pt").to(DEVICE)
    with torch.no_grad():
        text_feats = model.get_text_features(**inputs)
    text_feats = text_feats.cpu().numpy()

    # Normalize for cosine similarity
    faiss.normalize_L2(text_feats)

    # Search
    D, I = index.search(text_feats, top_k)

    # Collect results
    results = []
    for i, prompt in enumerate(prompts):
        matches = []
        for score, clip_id in zip(D[i], I[i]):
            info = lookup[str(int(clip_id))]
            matches.append({
                "prompt": prompt,
                "file": info["video_file"],
                "start_time_sec": info["start_time_sec"],
                "clip_index": info["clip_index"],
                "similarity": float(score)
            })
        results.append(matches)
    return results

prompts = [
    "Videos of a man injured in the daytime. Smoke is rising in the background",
    "A clown eating a huge bowl of spagetti while riding a bicycle"
]
results = search_prompts(prompts, top_k=3)
for match_list in results:
    for match in match_list:
        print(f"Prompt: {match['prompt']}")
        print(f"  File: {match['file']}")
        print(f"  Start time: {match['start_time_sec']}s (clip index {match['clip_index']})")
        print(f"  Similarity: {match['similarity']:.4f}")
        print()